# Save LightGBM model in CatBoost format to use fast CatBoost appliers

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/catboost/tutorials/blob/master/apply_model/fast_light_gbm_applier.ipynb)

To save LightGBM in CatBoost format you need to convert LightGBM model to ONNX, and then to convert the model from ONNX to CatBoost.
+ Save LightGBM model in the ONNX format
+ Load the ONNX model into CatBoost using the load_model() method
+ Apply your model in CatBoost using the predict() method or save it as file and use with other appliers

Note, that this tutorial will only work for Python 3.* since onnxmltools 

Let us follow this scenario step-by-step for a LightGBM model.


Download the MSRank dataset and import the necessary packages:

In [2]:
from catboost import datasets, CatBoostRegressor

from lightgbm import LGBMRegressor

import onnxmltools
from onnxconverter_common import *


train_df, _ = datasets.msrank_10k()
X, Y = train_df[train_df.columns[1:]], train_df[train_df.columns[0]]


Build a model:

In [3]:
model = LGBMRegressor()
model.fit(X, Y)
predict = model.predict(X)
print(predict)

[1.30604501 1.60390655 0.35207384 ... 1.18672199 0.55631924 0.54655847]



Save the model in the ONNX format:

In [4]:
features_count = len(X.columns)
onnx_model = onnxmltools.convert_lightgbm(model, name='LightGBM', initial_types=[['input', FloatTensorType([0, features_count])]])
onnxmltools.utils.save_model(onnx_model, 'model.onnx')

The maximum opset needed by this model is only 1.
The maximum opset needed by this model is only 1.



Load the ONNX model into CatBoost and print predictions to make sure they are correct.

In [5]:
catboost_model = CatBoostRegressor()
catboost_model.load_model('model.onnx', format='onnx')
catboost_predict = catboost_model.predict(X)
print(catboost_predict)

[1.30604502 1.60390654 0.35207381 ... 1.18672202 0.55631925 0.54655849]


Save the model in CatBoost format for later use in C++, C#, Java or Python code with fast CatBoost applier.

In [7]:
catboost_model.save_model('model.bin')